In [1]:
using DataFrames, CSV
using JuMP, Gurobi
using LinearAlgebra
using JSON

In [72]:
config = JSON.parsefile("config.json")

existing_distance = Matrix(CSV.read("inputs/candidate_existing_distance.csv", DataFrame, header = true))
attraction_distance = Matrix(CSV.read("inputs/candidate_attraction_distance.csv", DataFrame, header = true))
demand_raw = Matrix(CSV.read("inputs/salem_demand.csv", DataFrame, header = true))
demand = demand_raw[:, 9]
attraction_values_raw = Matrix(CSV.read("inputs/attractions_weights.csv", DataFrame, header = true))
attraction_values = attraction_values_raw[:,3]
candidate_addresses_raw = Matrix(CSV.read("inputs/candidate_address.csv", DataFrame, header = true));


In [73]:
row_num =  zeros(size(candidate_addresses_raw,1), 1)
for i=1:size(candidate_addresses_raw,1)
    row_num[i] = i
end
row_num
candidate_addresses = [row_num candidate_addresses_raw]

142×5 Matrix{Any}:
   1.0  …  42.5074  -70.9075
   2.0     42.5077  -70.9046
   3.0     42.5076  -70.9025
   4.0     42.5076  -70.8996
   5.0     42.5075  -70.897
   6.0  …  42.5078  -70.8943
   7.0     42.5078  -70.8917
   8.0     42.5079  -70.8891
   9.0     42.508   -70.8867
  10.0     42.5063  -70.8872
  11.0  …  42.5051  -70.8834
  12.0     42.5047  -70.8738
  13.0     42.5096  -70.9077
   ⋮    ⋱           
 131.0  …  42.529   -70.9081
 132.0     42.5291  -70.9056
 133.0     42.5293  -70.9028
 134.0     42.5291  -70.9001
 135.0     42.5296  -70.898
 136.0  …  42.5295  -70.8926
 137.0     42.5295  -70.8921
 138.0     42.5295  -70.8895
 139.0     42.5296  -70.8869
 140.0     42.5297  -70.8858
 141.0  …  42.5294  -70.8813
 142.0     42.5297  -70.8791

In [43]:
demand

13-element Vector{Any}:
 102.27272727272727
 133.46666666666667
 154.45454545454547
 242.1818181818182
 226.69230769230768
 101.27272727272727
 296.55555555555554
  70.81818181818181
  48.18181818181818
  23.363636363636363
  17.181818181818183
   4.666666666666667
   8.181818181818182

In [47]:
existing_distance

142×13 Matrix{Int64}:
 1706  1121  3922  1908  3049  3031  2156  3526  2558  2557  2821  5104  2748
 2557  2032  2377  2801  2874  2649  1775  3350  2585  2419  1883  4928  2402
 2187  1662  2149  2431  2504  2279  1405  2980  2216  2050  1656  4559  2032
 2382  1857  2152  2626  2699  2474  1600  3175  2410  2245  1659  4753  2227
 2671  2448  1005  3158  2291   812  1231  2768  1995  1837   959  4346  1820
 2755  2532   787  3242  2243   594  1314  2719  2079  1921   901  4298  1590
 2676  2453   486  3162  2055   405  1712  2531  2000  1842   602  4109  1401
 2884  2661   279  3371  2263   614  1822  2739  2208  2050   763  4318  1610
 3014  2791   285  3501  2393   743  1952  2869  2338  2180   928  4447  1740
 3455  3232   293  3942  2834  1185  2411  3310  2779  2621   947  4888  2181
 3602  3379   441  4089  2981  1332  2559  3458  2926  2768  1094  5036  2328
 3575  3353   414  4062  2954  1305  2532  3431  2900  2742  1068  5009  2301
 1511   926  3727  1713  2855  2836  1961 

In [48]:
attraction_distance

142×22 Matrix{Int64}:
 2835  3162  4037  2612  2289  3096  …  2812  2724  3050  2724  1730  2599
 2660  2986  3861  2475  2317  2920     2636  2548  2874  2587  2258  2423
 2290  2616  3492  2105  1947  2551     2266  2178  2505  2217  1888  2053
 2485  2811  3686  2300  2142  2746     2461  2373  2699  2412  2083  2248
 2077  2403  3279  1893  1956  2338     2054  1966  2292  2005  2223  1841
 2029  2355  3231  1976  2040  2290  …  2005  1917  2244  2088  2306  1792
 1841  2167  3042  1897  1961  2101     1817  1729  2055  1926  2227  1604
 2049  2375  3251  2106  2169  2310     2025  1937  2264  2134  2436  1812
 2179  2505  3380  2235  2299  2440     2155  2067  2393  2264  2565  1942
 2620  2946  3822  2676  2740  2881     2596  2508  2834  2705  3006  2383
 2767  3093  3969  2824  2887  3028  …  2744  2655  2982  2852  3154  2530
 2740  3066  3942  2797  2860  3001     2717  2629  2955  2826  3127  2504
 2641  2967  3842  2417  2094  2901     2617  2529  2855  2529  1535  2404
   

In [39]:
attraction_values

22-element Vector{Any}:
 3179
 8542
  151
  354
 3028
 1657
 3369
 2968
 1645
  221
   10
  669
  214
  447
   28
  139
   64
  292
 2091
 1339
   74
  151

In [49]:
# parameters

N_CANDIDATE_STATIONS = size(existing_distance)[1]
N_NEW_STATIONS = config["N_NEW_STATIONS"]
DISTANCE_THRESHOLD = config["DISTANCE_THRESHOLD"]
N_CURRENT_STATIONS = size(demand)[1]
N_ATTRACTIONS = size(attraction_distance)[2]

22

In [4]:
###FIRST VERSION OF MODEL

model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "OutputFlag", 0) 

M = 1_000_000

# @variable(model, x[i=1:50, j=1:15] >= 0) # amount of waste transported
@variable(model, z[i=1:N_CANDIDATE_STATIONS], Bin) # z: binary variable to indicate whether or not station is selected

# @constraint(model, [i=1:50], sum(x[i, j] for j in 1:15) == q[i])
# @constraint(model, [i=1:50, j=1:15], x[i, j] <= M * z[j])

# add constraint that two candidate stations cannot be too close to each other or existing stations

@constraint(model, sum(z[i] for i in 1:N_CANDIDATE_STATIONS) == N_NEW_STATIONS)
@constraint(model, [i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS], distance[i, j] >=  1000 * z[i])

@objective(model, Max, sum(-distance[i, j] * demand[j] * z[i] for i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS))

optimize!(model)

optimal_cost = objective_value(model)
stations_chosen = value.(z)


println("Optimal stations: $stations_chosen")


# printing optimal cost
pretty_optimal_cost = round(optimal_cost)
println("Total Cost: $pretty_optimal_cost")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-16
Optimal stations: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
Total Cost: -3.04235e6


In [41]:
candidate_station_addresses[findall(x -> x==1, stations_chosen)]

5-element Vector{Any}:
 "Unnamed Road, Salem, MA 01970, USA"
 "91 Leach St, Salem, MA 01970, USA"
 "Building 4, Suite 160 Shetland Park, 45 Congress St, Salem, MA 01970, USA"
 "Unnamed Road, Salem, MA 01970, USA"
 "98 Collins St, Salem, MA 01970, USA"

In [29]:
findall(stations_chosen)

LoadError: TypeError: non-boolean (Float64) used in boolean context

**OPTIMIZATION FORMULATION:**

Sets:
- candidate stations $i\in \mathcal{I}=\{1,\ldots,n\}$
- existing stations $j\in \mathcal{J}=\{1,\ldots,m\}$
- attractions $k\in \mathcal{K}=\{1,\ldots,q\}$

Decision variable: $z_{i} $ = 1 if candidate station i is built, 0 otherwise

Parameters:
- $e_{ij} $ = distance between candidate station i and existing station j
- $a_{ik} $ = distance between candidate station i and attraction k
- $d_{j} $ = demand at existing station j
- $v_{k} $ = number of google reviews for attraction k

$$ \max \quad \sum_{i = 1}^{n} \sum_{j=1}^{m} \frac{1}{e_{ij}} * d_{j}z_i\ +
\sum_{i = 1}^{n} \sum_{k=1}^{q} \frac{1}{a_{ik}} * v_{k}z_i\\

\text{s.t.} \quad \sum_{i = 1}^{n} z_i <= 5\\

e_{ij} >= 800z_i \quad \forall i,j\\

z_i\in\{0,1\}

$$

In [50]:
###SECOND VERSION OF MODEL

model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "OutputFlag", 0) 

M = 1_000_000

# i - candidate stations
# j - current stations
# k - attractions

@variable(model, z[i=1:N_CANDIDATE_STATIONS], Bin) # z: binary variable to indicate whether or not station is selected

# add constraint that two candidate stations cannot be too close to each other or existing stations

@constraint(model, sum(z[i] for i in 1:N_CANDIDATE_STATIONS) <= N_NEW_STATIONS) #can add up to N new stations
@constraint(model, [i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS], existing_distance[i, j] >= DISTANCE_THRESHOLD * z[i]) #distance between existing and candidate stations must be at least distance threshold

@objective(model, Max, 
    sum((1/existing_distance[i, j]) * demand[j] * z[i] for i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS) +
    sum((1/attraction_distance[i, k]) * attraction_values[k] * z[i] for i=1:N_CANDIDATE_STATIONS, k=1:N_ATTRACTIONS))

optimize!(model)

optimal_cost = objective_value(model)
stations_chosen = value.(z)


println("Optimal stations: $stations_chosen")


# printing optimal cost
pretty_optimal_cost = round(optimal_cost)
println("Total Cost: $pretty_optimal_cost")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Optimal stations: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Total Cost: 183.0


In [74]:
candidate_addresses[findall(x -> x==1, stations_chosen)]

5-element Vector{Any}:
 106.0
 115.0
 117.0
 126.0
 127.0

In [77]:
candidate_addresses_raw[findall(x -> x==1, stations_chosen)]

5-element Vector{Any}:
 "23 Derby St, Salem, MA 01970, USA"
 "16 Briggs St, Salem, MA 01970, USA"
 "4 Fort Ave, Salem, MA 01970, USA"
 "13 Pleasant St, Salem, MA 01970, USA"
 "90 Webb St, Salem, MA 01970, USA"

5-element Vector{Any}:
 106.0
    "23 Derby St, Salem, MA 01970, USA"
    "(42.5247067, -70.8786058)"
  42.5247067
 -70.8786058